## Pre-setup

In [ ]:
# import the aetest module
from pyats import aetest
import logging
from genie.testbed import load
import argparse
import sys
import pprint

# set debug level DEBUG, INFO, WARNING
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# set pprint
pp = pprint.PrettyPrinter(indent = 2)

## Loading the testbed, connecting to the device

In [3]:
testbed = load('/home/anorsoni/Projets/2020-CAP-Altitude/pyats-check-os/testbed.yaml')

asr903_5 = testbed.devices["ASR903_5"]

asr903_5.connect(init_exec_commands=[],
                 init_config_commands=[],
                 log_stdout=False)

In [4]:
isis = asr903_5.parse('show isis neighbors')
pp.pprint(isis)

{ 'isis': { 'Altitude': { 'neighbors': { 'ASR903_3': { 'type': { 'L2': { 'circuit_id': '1E',
                                                                         'holdtime': '25',
                                                                         'interface': 'BD1',
                                                                         'ip_address': '100.100.35.3',
                                                                         'state': 'UP'}}},
                                         'ASR903_4': { 'type': { 'L2': { 'circuit_id': '09',
                                                                         'holdtime': '25',
                                                                         'interface': 'Te0/2/4',
                                                                         'ip_address': '100.100.45.4',
                                                                         'state': 'UP'}}}}}}}


## Setting the db

In [7]:
import sqlite3
isis_connection = sqlite3.connect('../sqlite/db/isis.db')
isis_cursor = isis_connection.cursor()

In [11]:
pp.pprint(isis)

{ 'isis': { 'Altitude': { 'neighbors': { 'ASR903_3': { 'type': { 'L2': { 'circuit_id': '1E',
                                                                         'holdtime': '25',
                                                                         'interface': 'BD1',
                                                                         'ip_address': '100.100.35.3',
                                                                         'state': 'UP'}}},
                                         'ASR903_4': { 'type': { 'L2': { 'circuit_id': '09',
                                                                         'holdtime': '25',
                                                                         'interface': 'Te0/2/4',
                                                                         'ip_address': '100.100.45.4',
                                                                         'state': 'UP'}}}}}}}


In [41]:
for tag in isis['isis']:

    for neighbor in isis['isis'][tag]['neighbors']:
        my_router = asr903_5.name
        neighbor = neighbor
        print(isis['isis'][tag]['neighbors'][neighbor])

        for type in isis['isis'][tag]['neighbors'][neighbor]['type']:
            type = type
            interface = isis['isis'][tag]['neighbors'][neighbor]['type'][type]['interface']
            ip_address = isis['isis'][tag]['neighbors'][neighbor]['type'][type]['ip_address']

            isis_tuple = []
            isis_tuple.extend((my_router, neighbor, type, interface, ip_address))

            # isis_cursor.execute(f'''INSERT INTO isis VALUES ({my_router}, {neighbor}, {type}, {interface}, {ip_address})''')
            isis_cursor.execute(f'''REPLACE INTO isis (MyRouter, Neighbor, Type, Interface, Ip) VALUES (?,?,?,?,?)''', isis_tuple)
            isis_connection.commit()

{'type': {'L2': {'interface': 'BD1', 'ip_address': '100.100.35.3', 'state': 'UP', 'holdtime': '25', 'circuit_id': '1E'}}}
{'type': {'L2': {'interface': 'Te0/2/4', 'ip_address': '100.100.45.4', 'state': 'UP', 'holdtime': '25', 'circuit_id': '09'}}}


In [43]:
isis_cursor.execute('''SELECT * FROM isis''')
print(isis_cursor.fetchall())
                    

[('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3'), ('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4'), ('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3'), ('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4'), ('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3'), ('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4'), ('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3'), ('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4'), ('ASR903_5', 'ASR903_3', 'L2', 'BD1', '100.100.35.3'), ('ASR903_5', 'ASR903_4', 'L2', 'Te0/2/4', '100.100.45.4')]
